In [1]:
%pip install -qU langchain duckduckgo-search langchain-community


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

search.invoke("Obama")

'snippet: Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) is the 44th president of the United States (2009-17) and the first African American to hold the office. Before winning the presidency, Obama represented Illinois in the U.S. Senate (2005-08)., title: Barack Obama | Biography, Parents, Education, Presidency, Books ..., link: https://www.britannica.com/biography/Barack-Obama, snippet: Michelle LaVaughn Robinson Obama [1] (née Robinson; born January 17, 1964) is an American attorney and author who served as the first lady of the United States from 2009 to 2017, being married to Barack Obama, the 44th president of the United States., title: Michelle Obama - Wikipedia, link: https://en.wikipedia.org/wiki/Michelle_Obama, snippet: Barack Obama served as the 44th president of the United States (2009-17) and was the first African American to hold that post. A member of the Democratic Party, Obama previously represented Illinois in the U.S. Senate from 2005 to 2008., title: Bar

In [12]:
import os
import getpass
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_core.tools import tool

# Initialize OpenAI (or Groq) API Key
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq:")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama3-8b-8192", model_provider="groq")

def is_simple(input_text: str) -> bool:
    """Determine whether the input is simple."""
    prompt = PromptTemplate(
        input_variables=["text"],
        template="""
        Given the following input: "{text}", determine whether it is simple.
        Reply with 'yes' for simple and 'no' for complex.
        """
    )  
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(text=input_text).strip().lower()
    return response == "yes"

def split_into_simple_queries(input_text: str) -> list:
    """Splits complex input into multiple simple queries for a search engine tool."""
    prompt = PromptTemplate(
        input_variables=["text"],
        template="""
        Break down the following complex question into multiple simpler search queries, break them down
        that can be used with a search engine, do not add extra text:
        "{text}"
        """
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run(text=input_text).strip().split('\n')

@tool
def process_query(input_text: str):
    """Decides if the query is simple or complex and processes accordingly for a search engine tool."""
    if is_simple(input_text):
        return {"is_simple": True,"queries": [input_text]}
    else:
        return {"is_simple": False ,"queries": split_into_simple_queries(input_text)}

# Example Usage
if __name__ == "__main__":
    user_input = "Explain quantum mechanics and its impact on modern technology."
    result = process_query(user_input)
    print("Final Output:", result)


Final Output: {'is_simple': False, 'queries': ['Here are the broken-down search queries:', '', '* "quantum mechanics definition"', '* "what is quantum mechanics"', '* "quantum mechanics principles"', '* "how does quantum mechanics work"', '* "impact of quantum mechanics on technology"', '* "modern technology using quantum mechanics"', '* "applications of quantum mechanics"', '* "quantum mechanics in computing"', '* "quantum mechanics in communication"', '* "quantum mechanics in medicine"']}


In [14]:
import os
import getpass
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_core.tools import tool

# Initialize OpenAI (or Groq) API Key
if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq:")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama3-8b-8192", model_provider="groq")

def is_searchable(input_text: str) -> bool:
    """Determine whether the input requires a search engine."""
    prompt = PromptTemplate(
        input_variables=["text"],
        template="""
        Determine if the following input requires real-time information from a search engine:
        
        Examples:
        - "What is the capital of France?" -> Not Searchable
        - "Who is the current president of the United States?" -> Searchable
        - "Explain quantum mechanics." -> Not Searchable
        - "What happened yesterday in the stock market of the USA?" -> Searchable
        - "Tell me about the history of the Roman Empire." -> Not Searchable
        - "What are the latest updates on AI regulation?" -> Searchable
        
        Given the input: "{text}", reply with 'yes' if it requires a search engine or 'no' if it does not.
        """
    )  
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(text=input_text).strip().lower()
    return response == "yes"


@tool
def process_query(input_text: str):
    """Determines if a query requires a search engine and processes accordingly."""
    if is_searchable(input_text):
        return {"requires_search": True, "query": [input_text]}
    else:
        return {"requires_search": False, "query": [input_text]}
    
user_input = "What is the colour of Irish flag?"
result = process_query(user_input)
print("Final Output:", result)

Final Output: {'requires_search': False, 'query': ['What is the colour of Irish flag?']}
